# Import Library

Reference: https://github.com/google-research/bert

In [124]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Library for XML parsing
import xml.etree.ElementTree as ET

# Data Processing

In [125]:
path = "training_t2"
training_dir = "TRAINING_DATA"
data_2017_path = "2017_cases"
data_2018_path = "2018_cases"
pos = "pos"
neg = "neg"

In [126]:
pos_2018_path = os.path.join(path, training_dir, data_2018_path, pos)
neg_2018_path = os.path.join(path, training_dir, data_2018_path, neg)

# List of all the file name:
pos_2018_list = os.listdir(pos_2018_path)
neg_2018_list = os.listdir(neg_2018_path)


In [128]:
data_list = []
total = 0
for file_name in pos_2018_list:
    file_path = os.path.join(pos_2018_path, file_name)
    tree = ET.parse(file_path)
    root = tree.getroot()
    for child in root:
        for grand_child in child:
            if grand_child.tag == "TEXT":
                text = grand_child.text.replace('\n', '')
                
                if not text.isspace():
                    data_list.append([total, 1, 'a', text])
                    total += 1
for file_name in neg_2018_list:
    file_path = os.path.join(neg_2018_path, file_name)
    tree = ET.parse(file_path)
    root = tree.getroot()
    for child in root:
        for grand_child in child:
            if grand_child.tag == "TEXT":
                text = grand_child.text.replace('\n', '')
                if not text.isspace():
                    data_list.append([total, 0, 'a', text])
                    total += 1

In [129]:
df = pd.DataFrame(data=data_list, columns=["index", "label", "alpha", "text"])
df_bert_train_val, df_bert_test = train_test_split(df, test_size=0.2)
df_berf_train, df_berf_val = train_test_split(df_bert_train_val, test_size=0.2)

In [131]:
df_bert_train.to_csv('bert/data/train.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('bert/data/test.tsv', sep='\t', index=False, header=False)
df_berf_val.to_csv('bert/data/dev.tsv', sep='\t', index=False, header=False)

In [ ]:
terminal_command = "python3 run_classifier.py  \
--task_name=cola  \
--do_train=true  \
--do_eval=true  \
--do_predict=true \
--data_dir=./data/ \
--vocab_file=./uncased_L-2_H-128_A-2/vocab.txt \
--bert_config_file=./uncased_L-2_H-128_A-2/bert_config.json \
--init_checkpoint=./uncased_L-2_H-128_A-2/bert_model.ckpt \
--max_seq_length=64 \
--train_batch_size=16 \
--learning_rate=2e-5 \
--num_train_epochs=2 \
--output_dir=./bert_output/ \
--do_lower_case=True"